# ТЗ:
Необходимо автоматизировать сборку и отправку отчета по ключевым метриками продукта "лента новостей".
Отчет должен состоять из двух частей:
- текст с информацией о значениях ключевых метрик за предыдущий день
- график с значениями метрик за предыдущие 7 дней

В отчете должны быть следующие метрики: 
- DAU 
- Просмотры
- Лайки
- CTR

Отчет должен приходить ежедневно в 11:00 в чат. 

Код сборки отчета должен хранится в репозитории GitLab и его сборка и отправка должна быть автоматизирована с помощью GitLab CI/CD.

In [1]:
# !pip install telegram
# !pip install python-telegram-bot
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import pandahouse
import loggins_and_passwords as connect
from IPython.display import display

## подготовка и исследование возможностей.

### взаимодействие с telegram

Создадим бота в телеге. Для этого сначала напишем боту @BotFather и создадим бота там. При успешном создании мы получим токен.
После этого, мы в целом можем уже отправлять сообщения от имени бота. Но есть одно "но" - отправлять мы можем не кому угодно, а только тем, что сам инициировал взаимодействие с ботом. Поэтому сначала в телеге найдем созданного бота, стартанем его, получим свой chat_id, а уже потом будем отправлять.
Кроме того, будем скрывать наш токен бота и наш идентификатор чата. Не гоже чтобы голыми концами такое торчала наружу. Для этого в настройках CI/CD пропишем переменные, которые будут соответствовать токену и идентификатору чату, а в коде будем передавать не абсолютные значения, а переменные, которые ссылаются на значения.

In [2]:
bot = telegram.Bot(token=connect.bot_token)#os.environ.get("bot_token"))

chat_id = connect.chat_id #os.environ.get("chat_id")


Попробуем теперь отправить сообщение

In [3]:
# отправим текстовое сообщение
bot.sendMessage(chat_id=chat_id, text='hello world')

Отлично, все получилось. Теперь попробуем отправить не текст, а, например, график. 
По идее - надо сохранять график в виде файла и затем отправлять. Однако, в этом случае нужно предусмотреть протокол очистки файловой системы от этих самых файлов с графиками. Это муторно, поэтому будем перенаправлять поток вывода и создавать файл с графиком не в файловой системе, а в буфере.  

In [4]:
# создадим график
x = np.arange(1,10,1)
y = np.random.choice(5, len(x))

sns.lineplot(x,y)
plt.title('Test plot')

# заведем файловый объект в буфере, в который будем сохранять график
plot_object = io.BytesIO() 

# сохраним график в файловый объект в буфере
plt.savefig(plot_object)
plot_object.name = 'test_plot.png'

# перенесем курсор из конца файлового объекта в начало, чтобы потом читать весь файл
plot_object.seek(0)
plt.close()

# отпраавим график
bot.sendPhoto(chat_id=chat_id, photo=plot_object)

/nfs/env/lib/python3.8/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


Отлично, график тоже приходит. Попробуем отправить файл с данными. Для этого сначала выгрузим датафрейм из базы, затем сохраним его в файле хранящемся в буфере и отправим.

Чтобы не светить пароли и коннекторы к базе - мы можем засунуть их в переменные CI/CD. Но, поскольку мы еще не пишем боевой код, а только разбираемся - засунем хост, логин и пароль просто в отдельный файл, чтобы была хоть какая-то защита.

In [5]:
connection = {
    'host': connect.host,         #os.environ.get("db_host"),
    'password': connect.password, #os.environ.get("db_password"),
    'user': connect.user          #os.environ.get("db_login")
}

query = '''select * from simulator_20220420.feed_actions where toDate(time) = today() limit 100'''

df = pandahouse.read_clickhouse(query, connection=connection)
    
file_object = io.StringIO()
df.to_csv(file_object)
file_object.name = 'test_file.csv'
file_object.seek(0)
bot.sendDocument(chat_id=chat_id, document=file_object)

Супер. Теперь мы умеем отправлять в телегу все что нужно. Можно приступать к работе.

## Собираем отчет

### Текст

В первой части отчета у нас толжен быть текст содержащий основные метрики: DAU, лайки, просмотры и CTR за предыдущий день. В теории можно выгружать только 1 день данных, но, поскольку во второй части нам нужны уже графики построенные по данным за последние 7 дней - будем выгружать сразу последние 7 дней и уже из этих данных брать вчерашние значения. Делая так мы уменьшим количество обращений к бд.

In [6]:
def db_to_dataframe(query, connection):
    """
    db_to_dataframe(query, connection=connection)
    возвращает объект DataFrame выгруженный из базы.
    Параметры подключения к базе определены в переменной connection
    """
    return pandahouse.read_clickhouse(query, connection=connection)

***DAU***

In [7]:

query = '''
SELECT 
    toStartOfDay(toDateTime(time)) AS day,
    count(DISTINCT user_id) AS val
FROM 
    simulator_20220420.feed_actions
WHERE 
    toStartOfDay(toDateTime(time))<toStartOfDay(now())
    and 
    toStartOfDay(toDateTime(time))>= now() - INTERVAL 8 DAY
GROUP BY 
    toStartOfDay(toDateTime(time))
ORDER BY 
    day DESC

'''

dau = db_to_dataframe(query, connection)

dau_yesterday = dau.val.head(1).values[0]





***Likes, Views, CTR***

In [8]:
query = '''
SELECT 
       toStartOfDay(toDateTime(time)) AS day,
       countIf(user_id, action='like') AS likes,
       countIf(user_id, action='view') AS views,
       likes/views*100 as ctr
FROM 
    simulator_20220420.feed_actions
WHERE 
    toStartOfDay(toDateTime(time))<toStartOfDay(now())
    and 
    toStartOfDay(toDateTime(time))>= now() - INTERVAL 8 DAY
GROUP BY
    toStartOfDay(toDateTime(time))
ORDER BY day DESC
'''

likes_views_ctr = db_to_dataframe(query, connection)

date_yesterday = likes_views_ctr.head(1).values[0][0]
likes_yesterday = likes_views_ctr.head(1).values[0][1]
views_yesterday = likes_views_ctr.head(1).values[0][2]
ctr_yesterday = likes_views_ctr.head(1).values[0][3]


теперь сформируем сообщение

In [20]:
msg = '''
    ______________________________
    Отчет за {date}:
    \t - всего просмотров: {views}
    \t - всего лайков: {likes}
    \t - CTR: {ctr}%
    '''.format(date = date_yesterday.date(), 
               views='{0:,}'.format(views_yesterday).replace(',', ' '), 
               likes='{0:,}'.format(likes_yesterday).replace(',', ' '), 
               ctr = np.round(ctr_yesterday,2))



### Графики

Поскольку мы уже выгрузили все данные - нам осталось только визуализировать и отправить. Для начала соберем данный в один датафрейм.

In [65]:
df = dau.rename(columns={'val':'dau'}).set_index('day').join(
    likes_views_ctr.set_index('day'), 
    how='left').reset_index().sort_values(by='day', ascending=True).copy()

Теперь создадим графики и сохраним их ка файловый объект в буфере.

In [68]:
fig, axes = plt.subplots(2, 2, sharex=True, figsize=(20,7))
fig.suptitle('Основные метрики за последние 7 дней')
axes[0][0].set_title('DAU')
axes[0][1].set_title('CTR')
axes[1][0].set_title('Views')
axes[1][1].set_title('Likes')

sns.lineplot(ax=axes[0][0], x=df.day.values, y=df.dau.values)
sns.lineplot(ax=axes[0][1], x=df.day.values, y=df.ctr.values)
sns.lineplot(ax=axes[1][0], x=df.day.values, y=df.likes.values)
sns.lineplot(ax=axes[1][1], x=df.day.values, y=df.views.values)


# заведем файловый объект в буфере, в который будем сохранять график
plot_object = io.BytesIO() 

# сохраним график в файловый объект в буфере
plt.savefig(plot_object)
plot_object.name = 'auto_report.png'

# перенесем курсор из конца файлового объекта в начало, чтобы потом читать весь файл
plot_object.seek(0)
plt.close()


В целом, на этом мы закончили сборку отчета. Осталось только отправить боту.

## Отправляем отчет

In [69]:
# отправляем текст с показателями за вчера
bot.sendMessage(chat_id=chat_id, text=msg)

# отпраавим графики с показателями за последние 7 дней
bot.sendPhoto(chat_id=chat_id, photo=plot_object)

## Автоматизируем отчет

Теперь, когда мы все научились делать локально - создадим python файл, в котором будем собираться и из которого будет отправляться наш отчет. Затем создадим yml файл, который будет запускать сборку и отправку нашего отчета автоматически, по заранее заданному рассписанию.

***сборка отчета***

код из следующего блока следует сохранить в отдельный файл с расширением .py


In [1]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import pandahouse

def report():

    # получим доступ к боту
    bot = telegram.Bot(token = os.environ.get("bot_token"))

    chat_id = os.environ.get("chat_id")


    connection = {
        'host': os.environ.get("db_host"),
        'password': os.environ.get("db_password"),
        'user': os.environ.get("db_login")
    }

    def db_to_dataframe(query, connection):
        """
        db_to_dataframe(query, connection=connection)
        возвращает объект DataFrame выгруженный из базы.
        Параметры подключения к базе определены в переменной connection
        """
        return pandahouse.read_clickhouse(query, connection=connection)

    query = '''
    SELECT 
        toStartOfDay(toDateTime(time)) AS day,
        count(DISTINCT user_id) AS val
    FROM 
        simulator_20220420.feed_actions
    WHERE 
        toStartOfDay(toDateTime(time))<toStartOfDay(now())
        and 
        toStartOfDay(toDateTime(time))>= now() - INTERVAL 8 DAY
    GROUP BY 
        toStartOfDay(toDateTime(time))
    ORDER BY 
        day DESC

    '''

    dau = db_to_dataframe(query, connection)

    dau_yesterday = dau.val.head(1).values[0]

    query = '''
    SELECT 
           toStartOfDay(toDateTime(time)) AS day,
           countIf(user_id, action='like') AS likes,
           countIf(user_id, action='view') AS views,
           likes/views*100 as ctr
    FROM 
        simulator_20220420.feed_actions
    WHERE 
        toStartOfDay(toDateTime(time))<toStartOfDay(now())
        and 
        toStartOfDay(toDateTime(time))>= now() - INTERVAL 8 DAY
    GROUP BY
        toStartOfDay(toDateTime(time))
    ORDER BY day DESC
    '''

    likes_views_ctr = db_to_dataframe(query, connection)

    date_yesterday = likes_views_ctr.head(1).values[0][0]
    likes_yesterday = likes_views_ctr.head(1).values[0][1]
    views_yesterday = likes_views_ctr.head(1).values[0][2]
    ctr_yesterday = likes_views_ctr.head(1).values[0][3]


    msg = '''
        ______________________________
        Отчет за {date}:
        \t - всего просмотров: {views}
        \t - всего лайков: {likes}
        \t - CTR: {ctr}%
        '''.format(date = date_yesterday.date(), 
                   views='{0:,}'.format(views_yesterday).replace(',', ' '), 
                   likes='{0:,}'.format(likes_yesterday).replace(',', ' '), 
                   ctr = np.round(ctr_yesterday,2))

    df = dau.rename(columns={'val':'dau'}).set_index('day').join(
        likes_views_ctr.set_index('day'), 
        how='left').reset_index().sort_values(by='day', ascending=True).copy()

    fig, axes = plt.subplots(2, 2, sharex=True, figsize=(20,7))
    fig.suptitle('Основные метрики за последние 7 дней')
    axes[0][0].set_title('DAU')
    axes[0][1].set_title('CTR')
    axes[1][0].set_title('Views')
    axes[1][1].set_title('Likes')

    sns.lineplot(ax=axes[0][0], x=df.day.values, y=df.dau.values)
    sns.lineplot(ax=axes[0][1], x=df.day.values, y=df.ctr.values)
    sns.lineplot(ax=axes[1][0], x=df.day.values, y=df.likes.values)
    sns.lineplot(ax=axes[1][1], x=df.day.values, y=df.views.values)


    # заведем файловый объект в буфере, в который будем сохранять график
    plot_object = io.BytesIO() 

    # сохраним график в файловый объект в буфере
    plt.savefig(plot_object)
    plot_object.name = 'auto_report.png'

    # перенесем курсор из конца файлового объекта в начало, чтобы потом читать весь файл
    plot_object.seek(0)
    plt.close()

    # отправляем текст с показателями за вчера
    bot.sendMessage(chat_id=chat_id, text=msg)

    # отпраавим графики с показателями за последние 7 дней
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)
    
try:
    report()
except Exception as e:
    print(e)

***файл настройки CI/CD***

код из следующего блока следует сохранить в файл .gitlab-ci.yml и положить в корень репозитория. Данный файл содержит алгоритм сборки. 

***N.B.:*** в целом, можно не выносить сборку отчета в отдельный файл и запускать ноутбук. Однако, поскольку в докер-образе отсутствует jupyter notebook - внутри джобы, необходимо будет вставить секцию before_script, в которой выполнять команду ```pip install jupyter notebook```

In [ ]:
image: cr.yandex/crp742p3qacifd2hcon2/practice-da:latest

stages:
    - init
    - run

job_test_report_1:
    stage: run
    only:
        - schedules
    script:
        - python operational_report.py


***настройка рассписаня***

После того, как все подготовительные этапы сделаны - необходимо задать рассписание по которому будет собираться и отправляться отчет. Для этого в gitlab необходимо перейти на вкладку CI/CD -> Shedules -> New Schedule  и задать кастомное рассписание вида ```* 11 * * *```(по тз - оправка должна быть каждый день в 11 утра). Также необходимо указать имя, чтобы было понятно для чего это рассписание и часовой пояс, чтобы отчет приходил в 11 утра по часовому поясу заказчика, а не Лондона.

***настройка переменных***

Ранее, при сборке отчета, в целях безопасности, мы заменяли наши абсолютные значения, такие как идентификатор чата или токен бота, на переменные окружения. Для того, чтобы в нашем окружении появились эти переменные - нужно их создать в gitlab. Переходим на вкладку settings->CI/CD->Variables->Expand->Add Variable. В поле Key указываем имя переменной, по которому мы обращаемся к ней (например "bot_token"), а в поле Value - ее значение (т.е. сам токен, который получен нами от @BotFather)